In [2]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
import pickle

from plot_3d_helpers import expanded_detector_boundary_points, detector_boundary_points
from plot_3d_helpers import fps_clustering_downsample, get_min_dists, energy_weighted_density_sampling


# Loading File

In [3]:
f = uproot.open("input_files/bdt_convert_superunified_bnb_ncpi0_full_spacepoints.root")


In [4]:
rse = f["wcpselection"]["T_eval"].arrays(["run", "subrun", "event"], library="np")

true_nu_vtx = f["wcpselection"]["T_eval"].arrays(["truth_vtxX", "truth_vtxY", "truth_vtxZ"], library="np")
true_nu_vtx = np.stack([true_nu_vtx["truth_vtxX"], true_nu_vtx["truth_vtxY"], true_nu_vtx["truth_vtxZ"]], axis=-1)

reco_nu_vtx = f["wcpselection"]["T_PFeval"].arrays(["reco_nuvtxX", "reco_nuvtxY", "reco_nuvtxZ"], library="np")
reco_nu_vtx = np.stack([reco_nu_vtx["reco_nuvtxX"], reco_nu_vtx["reco_nuvtxY"], reco_nu_vtx["reco_nuvtxZ"]], axis=-1)


In [22]:
"""
 ('mc_trackid', <TBranchElement 'mc_trackid' at 0x00016c143160>),
 ('mc_mother', <TBranchElement 'mc_mother' at 0x00016c143a60>),
 ('mc_pdg', <TBranchElement 'mc_pdg' at 0x00016c158400>),
 ('mc_E', <TBranchElement 'mc_E' at 0x00016c158d60>),
 ('mc_n_elastic', <TBranchElement 'mc_n_elastic' at 0x00016c1596c0>),
 ('mc_n_inelastic', <TBranchElement 'mc_n_inelastic' at 0x00016c15a020>),
 ('mc_vx', <TBranchElement 'mc_vx' at 0x00016c15a980>),
 ('mc_vy', <TBranchElement 'mc_vy' at 0x00016c15b2e0>),
 ('mc_vz', <TBranchElement 'mc_vz' at 0x00016c15bc40>),
 ('mc_endx', <TBranchElement 'mc_endx' at 0x00016c1745e0>),
 ('mc_endy', <TBranchElement 'mc_endy' at 0x00016c174f40>),
 ('mc_endz', <TBranchElement 'mc_endz' at 0x00016c1758a0>),
 ('mc_px', <TBranchElement 'mc_px' at 0x00016c176200>),
 ('mc_py', <TBranchElement 'mc_py' at 0x00016c176b60>),
 ('mc_pz', <TBranchElement 'mc_pz' at 0x00016c1774c0>),
 ('mc_end_p', <TBranchElement 'mc_end_p' at 0x00016c177e20>),
"""

print("looking at the geant truth information for a shower from Pandora file")

geant_dic = f["nuselection"]["NeutrinoSelectionFilter"].arrays(["mc_trackid", "mc_mother", "mc_pdg", "mc_vx", "mc_vy", "mc_vz"], library="np")

one_event_wc_geant_dic = {}

for k, v in geant_dic.items():
    one_event_wc_geant_dic[k] = v[61]

one_event_wc_geant_dic


looking at the geant truth information for a shower from Pandora file


{'mc_trackid': array([1, 2, 3, 4], dtype=int32),
 'mc_mother': array([0, 0, 0, 0], dtype=int32),
 'mc_pdg': array([  14,  111, 2112, 2212], dtype=int32),
 'mc_vx': array([122.75491, 122.75491, 122.75491, 122.75491], dtype=float32),
 'mc_vy': array([-72.09211, -72.09211, -72.09211, -72.09211], dtype=float32),
 'mc_vz': array([809.7606, 809.7606, 809.7606, 809.7606], dtype=float32)}

In [19]:
# finding the primary pi0
primary_pi0_id = -1
for i in range(len(one_event_wc_geant_dic["mc_pdg"])):
    if one_event_wc_geant_dic["mc_pdg"][i] == 111 and one_event_wc_geant_dic["mc_mother"][i] == 0:
        primary_pi0_id = one_event_wc_geant_dic["mc_trackid"][i]
        print("found the primary pi0 at index: ", i, " with mc_trackid: ", primary_pi0_id)
        break

# finding the daughters of the primary pi0
daughters_of_primary_pi0_ids = []
for i in range(len(one_event_wc_geant_dic["mc_trackid"])):
    if one_event_wc_geant_dic["mc_mother"][i] == primary_pi0_id:
        print("  daughter of primary pi0")
        print("    mc_trackid:", one_event_wc_geant_dic["mc_trackid"][i])
        print("    mc_pdg:", one_event_wc_geant_dic["mc_pdg"][i])
        print("    mc_vx:", one_event_wc_geant_dic["mc_vx"][i])
        print("    mc_vy:", one_event_wc_geant_dic["mc_vy"][i])
        print("    mc_vz:", one_event_wc_geant_dic["mc_vz"][i])
        daughters_of_primary_pi0_ids.append(one_event_wc_geant_dic["truth_id"][i])

print("daughters of primary pi0: ", daughters_of_primary_pi0_ids)

# finding all descendants of the first daughter of the primary pi0
first_daughter_and_descendants_ids = [daughters_of_primary_pi0_ids[0]]
first_daughter_descendants_pdgs = []
num_added = 1
while num_added != 0:
    #print("looping over particles")
    num_added = 0
    for i in range(len(one_event_wc_geant_dic["mc_trackid"])):
        if one_event_wc_geant_dic["mc_mother"][i] in first_daughter_and_descendants_ids and one_event_wc_geant_dic["mc_trackid"][i] not in first_daughter_and_descendants_ids:
            first_daughter_and_descendants_ids.append(one_event_wc_geant_dic["mc_trackid"][i])
            first_daughter_descendants_pdgs.append(one_event_wc_geant_dic["mc_pdg"][i])
            num_added += 1

# finding the descendants of the second daughter of the primary pi0
second_daughter_and_descendants_ids = [daughters_of_primary_pi0_ids[1]]
second_daughter_descendants_pdgs = []
num_added = 1
while num_added != 0:
    #print("looping over particles")
    num_added = 0
    for i in range(len(one_event_wc_geant_dic["mc_trackid"])):
        if one_event_wc_geant_dic["mc_mother"][i] in second_daughter_and_descendants_ids and one_event_wc_geant_dic["mc_trackid"][i] not in second_daughter_and_descendants_ids:
            second_daughter_and_descendants_ids.append(one_event_wc_geant_dic["mc_trackid"][i])
            second_daughter_descendants_pdgs.append(one_event_wc_geant_dic["mc_pdg"][i])
            num_added += 1

print("first daughter descendants pdgs: ", first_daughter_descendants_pdgs)
print("second daughter descendants pdgs: ", second_daughter_descendants_pdgs)



found the primary pi0 at index:  1  with mc_trackid:  2
daughters of primary pi0:  []


IndexError: list index out of range

In [ ]:
# re-formatting the spacepoints before downsampling

num_events = 70
# num_events = len(truth_vtx["truth_vtxX"])

spacepoints = f["wcpselection"]["T_spacepoints"].arrays(["Tcluster_spacepoints_x", 
                                                                 "Tcluster_spacepoints_y", 
                                                                 "Tcluster_spacepoints_z", 
                                                                 #"Tcluster_spacepoints_q",
                                                                 "Trec_spacepoints_x", 
                                                                 "Trec_spacepoints_y", 
                                                                 "Trec_spacepoints_z", 
                                                                 #"Trec_spacepoints_q",
                                                                 "TrueEDep_spacepoints_startx",
                                                                 "TrueEDep_spacepoints_starty",
                                                                 "TrueEDep_spacepoints_startz",
                                                                 "TrueEDep_spacepoints_endx",
                                                                 "TrueEDep_spacepoints_endy",
                                                                 "TrueEDep_spacepoints_endz",
                                                                 "TrueEDep_spacepoints_edep",
                                                                 ], 
                                                                 entry_start=0, entry_stop=num_events, library="np")

true_pi0_spacepoints = f["nuselection"]["NeutrinoSelectionFilter"].arrays(["pi0truth_gamma1_xpos",
                                                                           "pi0truth_gamma1_ypos",
                                                                           "pi0truth_gamma1_zpos",
                                                                           "pi0truth_gamma1_edep",
                                                                           "pi0truth_gamma2_xpos",
                                                                           "pi0truth_gamma2_ypos",
                                                                           "pi0truth_gamma2_zpos",
                                                                           "pi0truth_gamma2_edep",
                                                                           ], 
                                                                           entry_start=0, entry_stop=num_events, library="np")

Tcluster_spacepoints = []
Trec_spacepoints = []
TrueEDep_spacepoints = []
TrueEDep_spacepoints_edep = []
for event_i in range(num_events):

    # reconstructed spacepoints 
    Tcluster_spacepoints.append(np.stack([spacepoints["Tcluster_spacepoints_x"][event_i],
                                         spacepoints["Tcluster_spacepoints_y"][event_i],
                                         spacepoints["Tcluster_spacepoints_z"][event_i]], axis=-1))
    Trec_spacepoints.append(np.stack([spacepoints["Trec_spacepoints_x"][event_i],
                                      spacepoints["Trec_spacepoints_y"][event_i],
                                      spacepoints["Trec_spacepoints_z"][event_i]], axis=-1))
    
    # true edep spacepoints
    starts = np.stack([spacepoints["TrueEDep_spacepoints_startx"][event_i],
                       spacepoints["TrueEDep_spacepoints_starty"][event_i],
                       spacepoints["TrueEDep_spacepoints_startz"][event_i]], axis=-1)
    midpoints = np.stack([(spacepoints["TrueEDep_spacepoints_startx"][event_i] + spacepoints["TrueEDep_spacepoints_endx"][event_i])/2,
                          (spacepoints["TrueEDep_spacepoints_starty"][event_i] + spacepoints["TrueEDep_spacepoints_endy"][event_i])/2,
                          (spacepoints["TrueEDep_spacepoints_startz"][event_i] + spacepoints["TrueEDep_spacepoints_endz"][event_i])/2], axis=-1)
    ends = np.stack([spacepoints["TrueEDep_spacepoints_endx"][event_i],
                     spacepoints["TrueEDep_spacepoints_endy"][event_i],
                     spacepoints["TrueEDep_spacepoints_endz"][event_i]], axis=-1)
    TrueEDep_spacepoints.append(np.concatenate([starts, midpoints, ends], axis=0))
    # assuming a third of the energy at the start, midpoint, and end
    TrueEDep_spacepoints_edep.append(np.concatenate([spacepoints["TrueEDep_spacepoints_edep"][event_i]/3,
                                                    spacepoints["TrueEDep_spacepoints_edep"][event_i]/3,
                                                    spacepoints["TrueEDep_spacepoints_edep"][event_i]/3], axis=0))
    
del spacepoints
del f


# Downsample Spacepoints

In [ ]:
close_to_reco_nu_vtx_threshold = 200
recalculate_downsampling = True

if recalculate_downsampling:
    downsampled_Tcluster_spacepoints = {}
    downsampled_Trec_spacepoints = {}
    downsampled_TrueEDep_spacepoints = {}
    downsampled_true_gamma1_EDep_spacepoints = {}
    downsampled_true_gamma2_EDep_spacepoints = {}
    for event_i in tqdm(range(num_events)):

        nearby_reco_nu_vtx_indices = np.where(np.sqrt((Tcluster_spacepoints[event_i][:, 0] - reco_nu_vtx[event_i][0])**2
                                                    + (Tcluster_spacepoints[event_i][:, 1] - reco_nu_vtx[event_i][1])**2
                                                    + (Tcluster_spacepoints[event_i][:, 2] - reco_nu_vtx[event_i][2])**2) < close_to_reco_nu_vtx_threshold)[0]
        Tcluster_spacepoints_near_reco_nu_vtx = Tcluster_spacepoints[event_i][nearby_reco_nu_vtx_indices, :]
        downsampled_Tcluster_spacepoints[event_i] = fps_clustering_downsample(Tcluster_spacepoints_near_reco_nu_vtx, 500)

        downsampled_Trec_spacepoints[event_i] = fps_clustering_downsample(Trec_spacepoints[event_i], 200)

        downsampled_TrueEDep_spacepoints[event_i] = energy_weighted_density_sampling(TrueEDep_spacepoints[event_i], TrueEDep_spacepoints_edep[event_i], 500)

        #downsampled_true_gamma1_EDep_spacepoints[event_i] = energy_weighted_density_sampling(true_gamma1_EDep_spacepoints[event_i], true_gamma1_EDep_spacepoints_edep[event_i], 500)
        #downsampled_true_gamma2_EDep_spacepoints[event_i] = energy_weighted_density_sampling(true_gamma2_EDep_spacepoints[event_i], true_gamma2_EDep_spacepoints_edep[event_i], 500)

    with open("downsampled_spacepoints.pkl", "wb") as f:
        pickle.dump((downsampled_Tcluster_spacepoints, downsampled_Trec_spacepoints, downsampled_TrueEDep_spacepoints, downsampled_true_gamma1_EDep_spacepoints, downsampled_true_gamma2_EDep_spacepoints), f)
else:
    with open("downsampled_spacepoints.pkl", "rb") as f:
        downsampled_Tcluster_spacepoints, downsampled_Trec_spacepoints, downsampled_TrueEDep_spacepoints = pickle.load(f)


  0%|          | 0/70 [00:00<?, ?it/s]

points:  [[220.3394928    9.12491035 513.10168457]
 [220.33963013   9.12218475 513.1027832 ]
 [220.34103394   9.09428883 513.11401367]
 ...
 [216.24050903 -22.01823997 515.36669922]
 [216.23884583 -22.01652718 515.36553955]
 [216.23872375 -22.01630211 515.36566162]]
len(points):  78396
points:  [218.22162    -3.6083694 513.3529   ]
len(points):  3


TypeError: object of type 'numpy.float32' has no len()

# Categorize Spacepoints

In [6]:
len(downsampled_Tcluster_spacepoints)

70

In [7]:
real_nu_reco_nu_downsampled_spacepoints = []
real_nu_reco_cosmic_downsampled_spacepoints = []
real_cosmic_reco_nu_downsampled_spacepoints = []
real_cosmic_reco_cosmic_downsampled_spacepoints = []

#real_gamma1_downsampled_spacepoints = []
#real_gamma2_downsampled_spacepoints = []

close_to_true_nu_spacepoint_threshold = 5
close_to_reco_nu_spacepoint_threshold = 5
close_to_true_gamma_spacepoint_threshold = 5

for event_i in range(num_events):

    if len(downsampled_Tcluster_spacepoints[event_i]) == 0:
        real_nu_reco_cosmic_downsampled_spacepoints.append(np.empty((0, 3)))
        real_nu_reco_nu_downsampled_spacepoints.append(np.empty((0, 3)))
        real_cosmic_reco_nu_downsampled_spacepoints.append(np.empty((0, 3)))
        real_cosmic_reco_cosmic_downsampled_spacepoints.append(np.empty((0, 3)))
        #real_gamma1_downsampled_spacepoints.append(np.empty((0, 3)))
        #real_gamma2_downsampled_spacepoints.append(np.empty((0, 3)))
        continue

    # for T_cluster spacepoints, noting distances to true nu and reco nu spacepoints, and which are close to the reco nu vtx
    min_truth_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_TrueEDep_spacepoints[event_i][:, :3])
    min_reco_nu_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_Trec_spacepoints[event_i][:, :3])
    min_true_gamma1_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_true_gamma1_EDep_spacepoints[event_i][:, :3])
    min_true_gamma2_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_true_gamma2_EDep_spacepoints[event_i][:, :3])

    # assign features to spacepoints here
    close_to_truth_indices = np.where(min_truth_dists < close_to_true_nu_spacepoint_threshold)[0]
    far_from_truth_indices = np.where(min_truth_dists >= close_to_true_nu_spacepoint_threshold)[0]
    close_to_reco_nu_indices = np.where(min_reco_nu_dists < close_to_reco_nu_spacepoint_threshold)[0]
    far_from_reco_nu_indices = np.where(min_reco_nu_dists >= close_to_reco_nu_spacepoint_threshold)[0]
    #close_to_true_gamma1_indices = np.where(min_true_gamma1_dists < close_to_true_gamma_spacepoint_threshold)[0]
    #close_to_true_gamma2_indices = np.where(min_true_gamma2_dists < close_to_true_gamma_spacepoint_threshold)[0]

    # categorize spacepoints here
    real_nu_reco_nu_indices = np.intersect1d(close_to_reco_nu_indices, close_to_truth_indices)
    real_nu_reco_cosmic_indices = np.intersect1d(far_from_reco_nu_indices, close_to_truth_indices)
    real_cosmic_reco_nu_indices = np.intersect1d(close_to_reco_nu_indices, far_from_truth_indices)
    real_cosmic_reco_cosmic_indices = np.intersect1d(far_from_reco_nu_indices, far_from_truth_indices)
    #real_gamma1_indices = close_to_true_gamma1_indices
    #real_gamma2_indices = close_to_true_gamma2_indices

    real_nu_reco_cosmic_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_nu_reco_cosmic_indices, :])
    real_nu_reco_nu_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_nu_reco_nu_indices, :])
    real_cosmic_reco_nu_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_cosmic_reco_nu_indices, :])
    real_cosmic_reco_cosmic_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_cosmic_reco_cosmic_indices, :])
    #real_gamma1_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_gamma1_indices, :])
    #real_gamma2_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_gamma2_indices, :])


# Plot in 3D

In [ ]:
include_non_downsampled_points = True

index = 61

run = rse["run"][index]
subrun = rse["subrun"][index]
event = rse["event"][index]

print("run, subrun, event:", run, subrun, event)


fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scene'}]])

# these are only added to set the camera at a better position
fig.add_trace(go.Scatter3d(
    x=expanded_detector_boundary_points[:, 2],
    y=expanded_detector_boundary_points[:, 0],
    z=expanded_detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=0.2,
        color='black',
        opacity=0.8
    ),
    name='Expanded TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=detector_boundary_points[:, 2],
    y=detector_boundary_points[:, 0],
    z=detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color='black',
        opacity=0.8
    ),
    name='TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=[reco_nu_vtx[index][2]],
    y=[reco_nu_vtx[index][0]],
    z=[reco_nu_vtx[index][1]],
    mode='markers',
    marker=dict(size=10, color='purple', opacity=1),
    name='Reco Neutrino Vertex',
    visible='legendonly'
))

fig.add_trace(go.Scatter3d(
    x=[true_nu_vtx[index][2]],
    y=[true_nu_vtx[index][0]],
    z=[true_nu_vtx[index][1]],
    mode='markers',
    marker=dict(size=10, color='green', opacity=1),
    name='True Neutrino Vertex',
    visible='legendonly'

))


if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=Tcluster_spacepoints[index][:, 2],
        y=Tcluster_spacepoints[index][:, 0],
        z=Tcluster_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color="blue",
            opacity=0.8
        ),
        name='Tcluster Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_Tcluster_spacepoints[index][:, 2],
    y=downsampled_Tcluster_spacepoints[index][:, 0],
    z=downsampled_Tcluster_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color="blue",
        opacity=0.8
    ),
    name='Downsampled Tcluster Spacepoints',
    visible='legendonly'
))

if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=Trec_spacepoints[index][:, 2],
        y=Trec_spacepoints[index][:, 0],
        z=Trec_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color='red',
            opacity=0.8
        ),
        name='Trec Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_Trec_spacepoints[index][:, 2],
    y=downsampled_Trec_spacepoints[index][:, 0],
    z=downsampled_Trec_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color='red',
        opacity=0.8
    ),
    name='Downsampled Trec Spacepoints',
    visible='legendonly'
))

if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=TrueEDep_spacepoints[index][:, 2],
        y=TrueEDep_spacepoints[index][:, 0],
        z=TrueEDep_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color='orange',
            opacity=0.8
        ),
        name='TrueEDep Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_TrueEDep_spacepoints[index][:, 2],
    y=downsampled_TrueEDep_spacepoints[index][:, 0],
    z=downsampled_TrueEDep_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='orange',
        opacity=0.8
    ),
    name='Downsampled TrueEDep Spacepoints',
    visible='legendonly'
))

fig.add_trace(go.Scatter3d(
    x=real_nu_reco_nu_downsampled_spacepoints[index][:, 2],
    y=real_nu_reco_nu_downsampled_spacepoints[index][:, 0],
    z=real_nu_reco_nu_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='orange',
        opacity=0.8
    ),
    name='Real Nu Reco Nu Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 2],
    y=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 0],
    z=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='red',
        opacity=0.8
    ),
    name='Real Nu Reco Cosmic Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 2],
    y=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 0],
    z=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='brown',
        opacity=0.8
    ),
    name='Real Cosmic Reco Nu Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 2],
    y=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 0],
    z=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='blue',
        opacity=0.8
    ),
    name='Real Cosmic Reco Cosmic Spacepoints',
))

"""
fig.add_trace(go.Scatter3d(
    x=real_gamma1_downsampled_spacepoints[index][:, 2],
    y=real_gamma1_downsampled_spacepoints[index][:, 0],
    z=real_gamma1_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='lightgreen',
        opacity=0.8
    ),
    name='Real Gamma 1 Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_gamma2_downsampled_spacepoints[index][:, 2],
    y=real_gamma2_downsampled_spacepoints[index][:, 0],
    z=real_gamma2_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='green',
        opacity=0.8
    ),
    name='Real Gamma 2 Spacepoints',
))
"""

fig.update_layout(
    scene=dict(
        xaxis_title='z',
        yaxis_title='x',
        zaxis_title='y',
        aspectratio=dict(
            x=5,
            y=3,
            z=1
        ),
    ),
    width=2000,
    height=1200,
    autosize=False,
    scene_camera=dict(
        eye=dict(x=-1.5, y=-1.5, z=1.5)
    )
)

fig.show(renderer="browser")


run, subrun, event: 20082 19 990
